# TSA Chapter 3: ADF vs KPSS Test Comparison[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch3/TSA_ch3_adf_kpss/TSA_ch3_adf_kpss.ipynb)Comparison of ADF and KPSS tests: reversed null hypotheses. ADF H0=unit root, KPSS H0=stationarity.

In [ ]:
!pip install numpy pandas matplotlib statsmodels scipy -q

In [ ]:
import osimport numpy as npimport pandas as pdimport matplotlib.pyplot as pltfrom statsmodels.tsa.stattools import acf, pacf, adfuller, kpssimport warningswarnings.filterwarnings('ignore')COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']plt.rcParams.update({    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',    'legend.framealpha': 0, 'legend.edgecolor': 'none',})np.random.seed(42)def save_chart(fig, name):    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)    print(f'Saved: {name}')

In [ ]:
# ADF vs KPSS test comparisonn = 300# Generate a stationary AR(1) and a non-stationary random walknp.random.seed(42)eps = np.random.normal(0, 1, n)# Stationary AR(1) with phi=0.7ar1 = np.zeros(n)for t in range(1, n):    ar1[t] = 0.7 * ar1[t-1] + eps[t]# Random walkrw = np.cumsum(np.random.normal(0, 1, n))fig, axes = plt.subplots(2, 2, figsize=(10, 6))# Plot the two seriesaxes[0, 0].plot(ar1, color=BLUE, linewidth=1.0)axes[0, 0].set_title('Stationary AR(1), $\\phi=0.7$', fontweight='bold')axes[0, 0].set_ylabel(r'$Y_t$')axes[0, 1].plot(rw, color=RED, linewidth=1.0)axes[0, 1].set_title('Non-stationary Random Walk', fontweight='bold')# Run tests on both seriesseries_list = [('AR(1)', ar1), ('Random Walk', rw)]colors = [BLUE, RED]# Create summary table as bar charttest_results = []for name, series in series_list:    adf_stat, adf_p, *_ = adfuller(series, autolag='AIC')    kpss_stat, kpss_p, *_ = kpss(series, regression='c', nlags='auto')    test_results.append((name, adf_p, kpss_p))labels = [r[0] for r in test_results]adf_pvals = [r[1] for r in test_results]kpss_pvals = [r[2] for r in test_results]x = np.arange(len(labels))width = 0.3axes[1, 0].bar(x - width/2, adf_pvals, width, color=GREEN, alpha=0.8, label='ADF p-value')axes[1, 0].bar(x + width/2, kpss_pvals, width, color=ORANGE, alpha=0.8, label='KPSS p-value')axes[1, 0].axhline(0.05, color=RED, linestyle='--', linewidth=1, label='5% significance')axes[1, 0].set_xticks(x)axes[1, 0].set_xticklabels(labels)axes[1, 0].set_title('P-values Comparison', fontweight='bold')axes[1, 0].set_ylabel('P-value')axes[1, 0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.22), ncol=3, frameon=False)# Decision matrixdecisions = [    ['ADF rejects\nKPSS fails to reject', 'Stationary'],    ['ADF fails to reject\nKPSS rejects', 'Unit root'],    ['Both reject', 'Inconclusive'],    ['Neither rejects', 'Inconclusive'],]axes[1, 1].axis('off')table = axes[1, 1].table(    cellText=[['ADF: reject, KPSS: not reject', 'Stationary'],              ['ADF: not reject, KPSS: reject', 'Unit root'],              ['Both reject', 'Inconclusive'],              ['Neither rejects', 'Inconclusive']],    colLabels=['Test Outcome', 'Conclusion'],    loc='center', cellLoc='center',)table.auto_set_font_size(False)table.set_fontsize(9)table.scale(1, 1.5)for key, cell in table.get_celld().items():    cell.set_edgecolor(GRAY)    if key[0] == 0:        cell.set_facecolor('#E8E8E8')        cell.set_text_props(fontweight='bold')    else:        cell.set_facecolor('none')axes[1, 1].set_title('Decision Matrix', fontweight='bold', pad=20)plt.tight_layout()save_chart(fig, 'sem3_adf_kpss')plt.show()